# 🦾 MILESTONE 3: COMPLETE AVENGERS GUARD SYSTEM
## Full Integration with LLM, Sound Effects, and Notifications

**Objective**: Integrate all components into a working security system

**Features**:
- ✅ Voice activation (Milestone 1)
- ✅ Face recognition (Milestone 2)
- ✅ LLM-powered dialogue (NEW)
- ✅ 4-level escalation (NEW)
- ✅ Sound effects per agent (NEW)
- ✅ Telegram notifications (NEW)
- ✅ Complete pipeline

**Demo Flow**:
1. Voice: "Jarvis, guard my room"
2. Face detection starts
3. Intruder detected → Agent responds
4. Auto-escalation with sound effects
5. Phone notification sent

In [1]:
import sys
sys.path.append('.')

In [2]:
# FIX: Add parent directory to path
import sys
import os
from pathlib import Path
sys.path.insert(0, str(Path.cwd()))
sys.path.insert(0, str(Path.cwd().parent))

In [3]:

# Cell 2: Install Dependencies
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("📦 Installing Milestone 3 dependencies...")
    # !pip install -q google-generativeai python-dotenv pygame requests pillow
else:
    print("💻 Running locally. Ensure all dependencies installed.")

# Cell 3: Import All Components
import sys
import os
from pathlib import Path

# Add src to path if not already
if str(Path.cwd() / 'src') not in sys.path:
    sys.path.insert(0, str(Path.cwd() / 'src'))

# Core imports
import cv2
import numpy as np
import time
from datetime import datetime
from typing import Optional, Dict, List

# Import our modules from previous milestones
from src.agents.agent_manager import AgentManager
from src.agents.base_agent import ThreatLevel
from src.core.state_machine import EscalationStateMachine, SystemState

# Import new Milestone 3 components
from src.audio.sound_effects import SoundEffectsManager
from src.notifications.telegram_bot import TelegramNotifier
from src.dialogue.llm_manager import LLMManager, AgentLLMIntegration

from src.integration.milestone2_classes import FaceEnrollmentSystem, FaceRecognitionEngine
from src.integration.milestone1_classes import AudioManager

print("✅ All imports successful!")

💻 Running locally. Ensure all dependencies installed.
pygame 2.6.1 (SDL 2.28.4, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
💻 Running locally. Ensure requirements.txt is installed.
✅ All imports successful!
✅ Directories created at data
📝 Enrollment System ready!
🔍 Recognition Engine ready!
🔍 VALIDATING MILESTONE 2 REQUIREMENTS

📂 Database loaded: 4 persons
✅ Face detection implemented
✅ Face recognition with embeddings
✅ Enrollment system (photo & webcam)
✅ Trusted persons database
✅ Real-time recognition engine
✅ Welcome messages for trusted users
✅ Intruder detection and warnings
✅ Recognition logging

🎯 MILESTONE 2 STATUS: COMPLETE ✅

👥 Enrolled persons: 4

👥 ENROLLED TRUSTED PERSONS
  • Mohith (owner)
    Enrolled: 2025-10-04
    Recognitions: 0
  • Damodar (roommate)
    Enrolled: 2025-09-30
    Recognitions: 0
  • Piyush (Friend)
    Enrolled: 2025-09-30
    Recognitions: 0
  • Arnav (Friend)
    Enrolled: 2025-10-06
    Recognitions: 0

In [4]:

# 
# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

print("✅ .env file loaded")
#  Cell 4: Configuration Class
class AvengersGuardConfig:
    """Complete system configuration"""
    
    # API Keys (set these or use .env file)
    GEMINI_API_KEY = os.getenv("GEMINI_API_KEY", "")
    TELEGRAM_BOT_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN", "")
    TELEGRAM_CHAT_ID = os.getenv("TELEGRAM_CHAT_ID", "")
    
    # Feature toggles
    ENABLE_LLM = True  # Use LLM for dynamic responses
    ENABLE_SOUND_FX = True  # Play sound effects
    ENABLE_NOTIFICATIONS = True  # Send Telegram notifications
    ENABLE_RECORDING = False  # Record intruder videos (optional)
    
    # System settings
    FRAME_SKIP = 3  # Process every Nth frame for performance
    RECOGNITION_CONFIDENCE = 0.6  # Face recognition threshold
    
    # Escalation timing (seconds)
    ESCALATION_TIMES = {
        1: 0,   # Level 1: Immediate
        2: 10,  # Level 2: After 10 seconds
        3: 25,  # Level 3: After 25 seconds
        4: 45   # Level 4: After 45 seconds
    }
    
    # Sound settings
    SOUND_VOLUME = 0.7  # 0.0 to 1.0
    
    # Notification settings
    NOTIFY_ON_LEVEL = 3  # Send phone notification at this level
    
    @staticmethod
    def validate():
        """Check if required configurations are set"""
        issues = []
        
        if AvengersGuardConfig.ENABLE_LLM and not AvengersGuardConfig.GEMINI_API_KEY:
            issues.append("GEMINI_API_KEY not set (LLM will use fallback responses)")
        
        if AvengersGuardConfig.ENABLE_NOTIFICATIONS:
            if not AvengersGuardConfig.TELEGRAM_BOT_TOKEN:
                issues.append("TELEGRAM_BOT_TOKEN not set (notifications disabled)")
            if not AvengersGuardConfig.TELEGRAM_CHAT_ID:
                issues.append("TELEGRAM_CHAT_ID not set (notifications disabled)")
        
        if issues:
            print("⚠️  Configuration Issues:")
            for issue in issues:
                print(f"   • {issue}")
            print("\n💡 See setup guides below")
        else:
            print("✅ Configuration validated!")
        
        return len(issues) == 0

config = AvengersGuardConfig()
config.validate()

✅ .env file loaded
✅ Configuration validated!


True

In [5]:
# Cell 5: Complete Guard System Class
class AvengersGuardSystem:
    """
    Complete integrated guard system
    Combines all components into one cohesive system
    """
    
    def __init__(self, config: AvengersGuardConfig):
        """Initialize complete system"""
        self.config = config
        
        print("\n🚀 Initializing Avengers Guard System...")
        print("="*60)
        
        # Initialize state machine
        self.state_machine = EscalationStateMachine()
        print("✅ State machine initialized")
        
        # Initialize agent manager
        self.agent_manager = AgentManager(rotation_mode="threat_based")
        self.current_agent = "jarvis"
        self.agent_manager.set_active_agent(self.current_agent)
        print("✅ Agent manager initialized (5 agents ready)")
        
        # Initialize sound effects
        self.sound_fx = None
        if config.ENABLE_SOUND_FX:
            self.sound_fx = SoundEffectsManager(volume=config.SOUND_VOLUME)
            print("✅ Sound effects manager initialized")
        
        # Initialize Telegram notifier
        self.notifier = None
        if config.ENABLE_NOTIFICATIONS and config.TELEGRAM_BOT_TOKEN and config.TELEGRAM_CHAT_ID:
            self.notifier = TelegramNotifier(
                config.TELEGRAM_BOT_TOKEN,
                config.TELEGRAM_CHAT_ID
            )
            print("✅ Telegram notifier initialized")
        
        # Initialize LLM
        self.llm_integration = None
        if config.ENABLE_LLM and config.GEMINI_API_KEY:
            try:
                llm = LLMManager(provider="gemini", api_key=config.GEMINI_API_KEY)
                self.llm_integration = AgentLLMIntegration(llm)
                print("✅ LLM integration initialized (Gemini)")
            except Exception as e:
                print(f"⚠️  LLM initialization failed: {e}")
                print("   Will use pre-scripted responses")
        
        # Import face recognition components
        try:
            from src.integration.milestone2_classes import FaceEnrollmentSystem, FaceRecognitionEngine
            
            self.enrollment = FaceEnrollmentSystem()
            self.face_engine = FaceRecognitionEngine(self.enrollment)
            print("✅ Face recognition loaded")
        except Exception as e:
            print(f"⚠️  Face recognition not available: {e}")
            self.enrollment = None
            self.face_engine = None
        
        # Import speech recognition
        try:
            from src.integration.milestone1_classes import AudioManager
            #self.audio_manager = AudioManager()
            print("✅ Speech recognition loaded")
        except Exception as e:
            print(f"⚠️  Speech recognition not available: {e}")
            self.audio_manager = None
        
        # Runtime state
        self.is_running = False
        self.frame_count = 0
        self.last_response_time = {}
        
        print("="*60)
        print("🎉 System initialization complete!\n")
    
    # ... rest of the methods stay the same (activate, deactivate, process_frame, etc.)
    
    def activate(self):
        """Activate the guard system"""
        self.state_machine.activate()
        
        # Get activation message from current agent
        agent_name, message = self.agent_manager.get_activation_message()
        print(f"\n🤖 {agent_name}: {message}")
        
        # Play activation sound
        if self.sound_fx:
            self.sound_fx.play_agent_activation(self.current_agent)
        
        # Speak activation message
        if self.audio_manager:
            self.audio_manager.speak(message)
        
        # Send notification
        if self.notifier:
            self.notifier.send_activation_alert(agent_name)
        
        self.is_running = True
        print("✅ System ACTIVATED and monitoring\n")
    
    def deactivate(self):
        """Deactivate the guard system"""
        duration = self.state_machine.deactivate()
        
        # Get deactivation message
        agent_name, message = self.agent_manager.get_deactivation_message()
        print(f"\n🤖 {agent_name}: {message}")
        
        # Speak deactivation message
        if self.audio_manager:
            self.audio_manager.speak(message)
        
        # Send notification
        if self.notifier:
            self.notifier.send_deactivation_alert(int(duration))
        
        self.is_running = False
        print("✅ System DEACTIVATED\n")
        
        # Show statistics
        stats = self.state_machine.get_statistics()
        print(f"📊 Session Statistics:")
        print(f"   Duration: {int(duration)} seconds")
        print(f"   Intruders detected: {stats['total_intruders_detected']}")
        print(f"   Escalations: {stats['total_escalations']}")
        print(f"   Events logged: {stats['total_events']}")
    
    def process_frame(self, frame: np.ndarray) -> np.ndarray:
        """
        Process a video frame
        
        Args:
            frame: Video frame from webcam
        
        Returns:
            Annotated frame with detection boxes
        """
        self.frame_count += 1
        
        # Skip frames for performance
        if self.frame_count % self.config.FRAME_SKIP != 0:
            return frame
        
        # Use face recognition if available
        if self.face_engine:
            annotated_frame, detections = self.face_engine.process_frame(frame)
            
            # Process each detection
            for detection in detections:
                if detection['trusted']:
                    self._handle_trusted_person(detection)
                else:
                    self._handle_intruder(detection, frame)
            
            return annotated_frame
        
        return frame
    
    def _handle_trusted_person(self, detection: Dict):
        """Handle detection of trusted person"""
        person_name = detection['name']
        person = self.enrollment.trusted_persons[person_name]
        
        # Get greeting
        agent_name, greeting = self.agent_manager.get_greeting(
            person_name, 
            person.role
        )
        
        print(f"✅ {agent_name}: {greeting}")
        
        # Speak greeting (optional - might be annoying)
        # if self.audio_manager:
        #     self.audio_manager.speak(greeting)
        
        # Send welcome notification (optional)
        # if self.notifier:
        #     self.notifier.send_welcome_message(person_name, person.role)
    
    def _handle_intruder(self, detection: Dict, frame: np.ndarray):
        """Handle intruder detection"""
        # Create intruder ID based on location
        loc = detection['location']
        intruder_id = f"intruder_{loc[0]}_{loc[1]}"  # Simple ID
        
        # Process detection in state machine
        info = self.state_machine.process_detection(intruder_id, loc)
        
        threat_level = info['threat_level']
        time_present = info['time_present']
        
        # Check if we should respond (don't spam responses)
        should_respond = self._should_respond(intruder_id, threat_level)
        
        if should_respond:
            self._give_intruder_response(
                intruder_id, 
                threat_level, 
                time_present,
                frame
            )
    
    def _should_respond(self, intruder_id: str, threat_level: int) -> bool:
        """Determine if we should give a response now"""
        current_time = time.time()
        
        # Check last response time
        if intruder_id in self.last_response_time:
            time_since_last = current_time - self.last_response_time[intruder_id]
            
            # Response intervals based on threat level
            intervals = {1: 8, 2: 5, 3: 3, 4: 1}  # seconds
            required_interval = intervals.get(threat_level, 5)
            
            if time_since_last < required_interval:
                return False
        
        return True
    
    def _give_intruder_response(self, intruder_id: str, threat_level: int,
                                time_present: float, frame: np.ndarray):
        """Give response to intruder"""
        
        # Select agent based on threat level
        agent_map = {
            1: "jarvis",
            2: "captain_america",
            3: "hulk",
            4: "thor"
        }
        selected_agent = agent_map.get(threat_level, "jarvis")
        
        # Switch agent if needed
        if selected_agent != self.current_agent:
            self.current_agent = selected_agent
            self.agent_manager.set_active_agent(selected_agent)
            print(f"\n🔄 Switching to {selected_agent.upper()}")
        
        # Generate response
        if self.llm_integration and self.config.ENABLE_LLM:
            # Use LLM for dynamic response
            response = self.llm_integration.generate_intruder_response(
                agent_name=selected_agent,
                threat_level=threat_level,
                intruder_action=f"present for {int(time_present)} seconds",
                use_llm=True
            )
        else:
            # Use pre-scripted response
            from src.agents.base_agent import InteractionContext
            context = InteractionContext(
                person_name=None,
                is_trusted=False,
                threat_level=ThreatLevel(threat_level),
                interaction_count=threat_level,
                time_since_first_detection=time_present,
                previous_responses=[]
            )
            agent_name, response = self.agent_manager.get_intruder_response(context)
        
        # Display response
        print(f"\n🚨 {selected_agent.upper()} (Level {threat_level}): {response}")
        
        # Play sound effect
        if self.sound_fx:
            print(f"DEBUG: Calling sound for {selected_agent}, level {threat_level}")  # ADD
            self.sound_fx.play_agent_alert(selected_agent, threat_level)
            print(f"DEBUG: Sound call completed")  # ADD
        
        # Speak response
        if self.audio_manager:
            self.audio_manager.speak(response)
        
        # Send notification at certain threat level
        if self.notifier and threat_level >= self.config.NOTIFY_ON_LEVEL:
            self.notifier.send_intruder_alert(
                selected_agent,
                threat_level,
                image=frame
            )
        
        # Play alarm at max level
        if threat_level == 4 and self.sound_fx:
            self.sound_fx.play_alarm(duration=2.0)
        
        # Update last response time
        self.last_response_time[intruder_id] = time.time()
        
        # Log response
        self.state_machine.add_response(intruder_id, response)
    
    def run_monitoring(self, duration: int = 60):
        """
        Run live monitoring from webcam
        
        Args:
            duration: How long to run (seconds)
        """
        if not self.face_engine:
            print("❌ Face recognition not available. Cannot run monitoring.")
            return
        
        print(f"🎥 Starting live monitoring for {duration} seconds...")
        print("   Press 'q' to quit early\n")
        
        # Activate system
        self.activate()
        
        # Open webcam
        video_capture = cv2.VideoCapture(0)
        
        if not video_capture.isOpened():
            print("❌ Could not access webcam!")
            return
        
        start_time = time.time()
        
        try:
            while (time.time() - start_time) < duration and self.is_running:
                ret, frame = video_capture.read()
                if not ret:
                    continue
                
                # Process frame
                annotated_frame = self.process_frame(frame)
                
                # Cleanup old intruders
                self.state_machine.cleanup_old_intruders()
                
                # Display
                cv2.imshow('Avengers Guard - Press Q to quit', annotated_frame)
                
                # Check for quit
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
        
        finally:
            video_capture.release()
            cv2.destroyAllWindows()
            
            # Deactivate
            if self.is_running:
                self.deactivate()

print("✅ AvengersGuardSystem class ready!")

✅ AvengersGuardSystem class ready!


In [6]:
# Cell 6: Quick Setup Functions
def quick_setup_guide():
    """Display setup guide for all components"""
    print("\n" + "="*60)
    print("🚀 MILESTONE 3 SETUP GUIDE")
    print("="*60)
    
    print("\n1️⃣ FACE RECOGNITION (from Milestone 2)")
    print("   • Run: enrollment = FaceEnrollmentSystem()")
    print("   • Enroll yourself: enrollment.enroll_from_webcam('YourName', 'owner')")
    
    print("\n2️⃣ GOOGLE GEMINI API (for LLM)")
    print("   • Visit: https://makersuite.google.com/app/apikey")
    print("   • Get API key (free)")
    print("   • Set: os.environ['GEMINI_API_KEY'] = 'your_key'")
    
    print("\n3️⃣ TELEGRAM BOT (for notifications)")
    print("   • Message @BotFather in Telegram")
    print("   • Create bot, get token")
    print("   • Message @userinfobot for chat ID")
    print("   • Set: os.environ['TELEGRAM_BOT_TOKEN'] = 'your_token'")
    print("   • Set: os.environ['TELEGRAM_CHAT_ID'] = 'your_chat_id'")
    
    print("\n4️⃣ SOUND EFFECTS (optional)")
    print("   • Download from: zapsplat.com, freesound.org")
    print("   • Place in: sounds/agents/, sounds/system/")
    print("   • See SoundEffectsManager.print_setup_guide()")
    
    print("\n5️⃣ TEST SYSTEM")
    print("   • Run: system = AvengersGuardSystem(config)")
    print("   • Run: system.run_monitoring(duration=30)")
    
    print("\n" + "="*60 + "\n")


In [7]:

# Cell 7: Testing Functions
def test_sound_effects():
    """Test sound effects system"""
    print("🎵 Testing Sound Effects...\n")
    
    sfx = SoundEffectsManager()
    
    print("Available sounds:", len(sfx.available_sounds))
    
    if len(sfx.available_sounds) == 0:
        print("⚠️  No sound files found. See setup guide:")
        sfx.print_setup_guide()
    else:
        print("✅ Playing available sounds...")
        for sound_name in list(sfx.available_sounds.keys())[:3]:
            print(f"   🔊 {sound_name}")
            sfx.play_sound(sound_name, wait=True)
            time.sleep(0.5)

def test_telegram():
    """Test Telegram notifications"""
    print("📱 Testing Telegram...\n")
    
    token = config.TELEGRAM_BOT_TOKEN
    chat_id = config.TELEGRAM_CHAT_ID
    
    if not token or not chat_id:
        print("⚠️  Telegram not configured. See setup guide:")
        from src.notifications.telegram_bot import setup_telegram_bot
        setup_telegram_bot()
        return
    
    notifier = TelegramNotifier(token, chat_id)
    
    print("Sending test message...")
    success = notifier.send_message("🛡️ Avengers Guard Test Message!")
    
    if success:
        print("✅ Message sent! Check your Telegram")
    else:
        print("❌ Failed to send. Check token and chat_id")

def test_llm():
    """Test LLM integration"""
    print("🤖 Testing LLM...\n")
    
    if not config.GEMINI_API_KEY:
        print("⚠️  Gemini API key not set. See setup guide:")
        from src.dialogue.llm_manager import setup_llm_guide
        setup_llm_guide()
        return
    
    try:
        llm = LLMManager(provider="gemini", api_key=config.GEMINI_API_KEY)
        agent_llm = AgentLLMIntegration(llm)
        
        print("Testing JARVIS response...")
        response = agent_llm.generate_intruder_response(
            "jarvis", 
            threat_level=1,
            intruder_action="standing in doorway"
        )
        
        print(f"✅ JARVIS: {response}")
        
    except Exception as e:
        print(f"❌ LLM test failed: {e}")


In [8]:

# Cell 8: Main Demo Script
def run_complete_demo():
    """
    Run complete system demo
    Shows all features working together
    """
    print("\n" + "🦾 "*20)
    print("COMPLETE AVENGERS GUARD SYSTEM DEMO")
    print("🦾 "*20 + "\n")
    
    # Create system
    system = AvengersGuardSystem(config)
    
    # Run monitoring
    print("\n💡 Demo will run for 60 seconds")
    print("   • Show your face (should recognize you)")
    print("   • Have someone else enter (intruder)")
    print("   • Watch escalation happen!")
    print("   • Check your phone for notifications\n")
    
    input("Press ENTER to start demo...")
    
    system.run_monitoring(duration=60)
    
    print("\n✅ Demo complete!")

In [9]:

# Cell 9: Validation and Ready Check
print("\n" + "="*60)
print("🎯 MILESTONE 3 READY CHECK")
print("="*60 + "\n")

# Show what's ready
print("📋 Component Status:")
print(f"   {'✅' if config.GEMINI_API_KEY else '⚠️ '} LLM (Gemini)")
print(f"   {'✅' if config.TELEGRAM_BOT_TOKEN and config.TELEGRAM_CHAT_ID else '⚠️ '} Telegram Notifications")
print(f"   ✅ Sound Effects (download files)")
print(f"   ✅ Face Recognition (enroll faces)")
print(f"   ✅ State Machine")
print(f"   ✅ 5 Agent Personalities")

print("\n🚀 Quick Start:")
print("   1. quick_setup_guide() - See setup instructions")
print("   2. test_sound_effects() - Test sounds")
print("   3. test_telegram() - Test notifications")
print("   4. test_llm() - Test LLM")
print("   5. run_complete_demo() - Run full system!")

print("\n" + "="*60)


🎯 MILESTONE 3 READY CHECK

📋 Component Status:
   ✅ LLM (Gemini)
   ✅ Telegram Notifications
   ✅ Sound Effects (download files)
   ✅ Face Recognition (enroll faces)
   ✅ State Machine
   ✅ 5 Agent Personalities

🚀 Quick Start:
   1. quick_setup_guide() - See setup instructions
   2. test_sound_effects() - Test sounds
   3. test_telegram() - Test notifications
   4. test_llm() - Test LLM
   5. run_complete_demo() - Run full system!



In [10]:

# Cell 10: Example Usage
print("""
📖 EXAMPLE USAGE:

# Basic setup
>>> quick_setup_guide()

# Test components
>>> test_sound_effects()
>>> test_telegram()
>>> test_llm()

# Create and run system
>>> system = AvengersGuardSystem(config)
>>> system.run_monitoring(duration=60)

# Or run complete demo
>>> run_complete_demo()

Ready to guard your room! 🛡️
""")


📖 EXAMPLE USAGE:

# Basic setup
>>> quick_setup_guide()

# Test components
>>> test_sound_effects()
>>> test_telegram()
>>> test_llm()

# Create and run system
>>> system = AvengersGuardSystem(config)
>>> system.run_monitoring(duration=60)

# Or run complete demo
>>> run_complete_demo()

Ready to guard your room! 🛡️



In [11]:
quick_setup_guide()


🚀 MILESTONE 3 SETUP GUIDE

1️⃣ FACE RECOGNITION (from Milestone 2)
   • Run: enrollment = FaceEnrollmentSystem()
   • Enroll yourself: enrollment.enroll_from_webcam('YourName', 'owner')

2️⃣ GOOGLE GEMINI API (for LLM)
   • Visit: https://makersuite.google.com/app/apikey
   • Get API key (free)
   • Set: os.environ['GEMINI_API_KEY'] = 'your_key'

3️⃣ TELEGRAM BOT (for notifications)
   • Message @BotFather in Telegram
   • Create bot, get token
   • Message @userinfobot for chat ID
   • Set: os.environ['TELEGRAM_BOT_TOKEN'] = 'your_token'
   • Set: os.environ['TELEGRAM_CHAT_ID'] = 'your_chat_id'

4️⃣ SOUND EFFECTS (optional)
   • Download from: zapsplat.com, freesound.org
   • Place in: sounds/agents/, sounds/system/
   • See SoundEffectsManager.print_setup_guide()

5️⃣ TEST SYSTEM
   • Run: system = AvengersGuardSystem(config)
   • Run: system.run_monitoring(duration=30)




In [25]:
test_sound_effects()

🎵 Testing Sound Effects...

✅ Sound system initialized
✅ Loaded 15 sound effects
Available sounds: 15
✅ Playing available sounds...
   🔊 jarvis_activate
   🔊 cap_activate
   🔊 widow_activate


In [13]:
test_llm()

🤖 Testing LLM...

✅ Google Gemini initialized
Testing JARVIS response...
⚠️  Gemini error: 404 models/gemini-1.5-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.
✅ JARVIS: Good day. I don't believe we've been introduced. Might I ask who you are?


In [14]:
test_telegram()

📱 Testing Telegram...

✅ Telegram bot connected successfully!
Sending test message...
✅ Message sent! Check your Telegram


In [15]:
system = AvengersGuardSystem(config)


🚀 Initializing Avengers Guard System...
✅ State machine initialized
🤖 Active Agent: JARVIS
🤖 Active Agent: JARVIS
✅ Agent manager initialized (5 agents ready)
✅ Sound system initialized
✅ Loaded 15 sound effects
✅ Sound effects manager initialized
✅ Telegram bot connected successfully!
✅ Telegram notifier initialized
✅ Google Gemini initialized
✅ LLM integration initialized (Gemini)
📂 Database loaded: 4 persons
✅ Face recognition loaded
✅ Speech recognition loaded
🎉 System initialization complete!



In [17]:
system.run_monitoring(duration=60)

🎥 Starting live monitoring for 60 seconds...
   Press 'q' to quit early

🛡️  System ARMED

🤖 JARVIS: JARVIS online. Security protocol Alpha-1 engaged. Room is now under surveillance.


AttributeError: 'AvengersGuardSystem' object has no attribute 'audio_manager'

In [18]:
run_complete_demo()


🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 
COMPLETE AVENGERS GUARD SYSTEM DEMO
🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 


🚀 Initializing Avengers Guard System...
✅ State machine initialized
🤖 Active Agent: JARVIS
🤖 Active Agent: JARVIS
✅ Agent manager initialized (5 agents ready)
✅ Sound system initialized
✅ Loaded 15 sound effects
✅ Sound effects manager initialized
✅ Telegram bot connected successfully!
✅ Telegram notifier initialized
✅ Google Gemini initialized
✅ LLM integration initialized (Gemini)
📂 Database loaded: 4 persons
✅ Face recognition loaded
✅ Speech recognition loaded
🎉 System initialization complete!


💡 Demo will run for 60 seconds
   • Show your face (should recognize you)
   • Have someone else enter (intruder)
   • Watch escalation happen!
   • Check your phone for notifications

🎥 Starting live monitoring for 60 seconds...
   Press 'q' to quit early

🛡️  System ARMED

🤖 JARVIS: Security systems active. I assure you, nothing shall escape my notice.


AttributeError: 'AvengersGuardSystem' object has no attribute 'audio_manager'

In [19]:
import os
for file in os.listdir("sounds/agents"):
    print(file)

cap_shield.mp3
hulk_roar.mp3
hulk_smash.mp3
jarvis_activate.mp3
jarvis_alert.mp3
thor_mjolnir.mp3
thor_thunder.mp3
widow_stealth.mp3


In [20]:
system.sound_fx.play_sound("hulk_smash")

⚠️  Sound not found: hulk_smash


False

In [21]:
import pygame
from pathlib import Path

pygame.mixer.init()

# Test the problem file
file = Path("sounds/agents/hulk_smash.mp3")
print(f"File exists: {file.exists()}")
print(f"File size: {file.stat().st_size} bytes")

try:
    pygame.mixer.music.load(str(file))
    pygame.mixer.music.play()
    print("Playing hulk_smash...")
    import time
    time.sleep(3)
except Exception as e:
    print(f"Error: {e}")

File exists: True
File size: 71888 bytes
Playing hulk_smash...


In [22]:
# Test the system's sound calls specifically
from src.audio.sound_effects import SoundEffectsManager
import time

sfx = SoundEffectsManager()

print("Testing agent alerts with delays...")

# Test each agent alert exactly as system calls it
for agent in ["jarvis", "captain_america", "hulk", "thor"]:
    print(f"\nTesting {agent}...")
    success = sfx.play_agent_alert(agent, threat_level=3)
    print(f"  Result: {success}")
    time.sleep(3)  # Wait between tests

✅ Sound system initialized
✅ Loaded 15 sound effects
Testing agent alerts with delays...

Testing jarvis...
  Result: True

Testing captain_america...
  Result: True

Testing hulk...
  Result: True

Testing thor...
  Result: True


In [23]:
# Debug checklist
print("=== SYSTEM DEBUG ===\n")

print("1. Config check:")
print(f"   ENABLE_SOUND_FX = {config.ENABLE_SOUND_FX}")

print("\n2. System sound object:")
print(f"   system.sound_fx = {system.sound_fx}")
print(f"   Is None? {system.sound_fx is None}")

print("\n3. Available sounds:")
if system.sound_fx:
    print(f"   Total: {len(system.sound_fx.available_sounds)}")
    print(f"   hulk_alert in list? {'hulk_alert' in system.sound_fx.available_sounds}")

print("\n4. Test direct call:")
if system.sound_fx:
    result = system.sound_fx.play_agent_alert("hulk", 3)
    print(f"   Direct call result: {result}")

=== SYSTEM DEBUG ===

1. Config check:
   ENABLE_SOUND_FX = True

2. System sound object:
   system.sound_fx = <src.audio.sound_effects.SoundEffectsManager object at 0x000001E9A5AAE450>
   Is None? False

3. Available sounds:
   Total: 15
   hulk_alert in list? True

4. Test direct call:
   Direct call result: True


In [24]:
import numpy as np

# Fake an intruder detection
fake_frame = np.zeros((480, 640, 3), dtype=np.uint8)

system.activate()
system._give_intruder_response(
    intruder_id="test_intruder",
    threat_level=3,  # Hulk level
    time_present=25.0,
    frame=fake_frame
)

🛡️  System ARMED

🤖 JARVIS: JARVIS at your service. Perimeter defense protocols now active.


AttributeError: 'AvengersGuardSystem' object has no attribute 'audio_manager'